In [ ]:
#import pkg_resources

#pkg_resources.require("numpy==1.21")

import numpy as np
import matplotlib.pylab as plt
import uproot
import awkward as ak

import vector

import sys

vector.register_awkward()

#from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

import coffea

from coffea.nanoevents import NanoEventsFactory
#from coffea.nanoevents import NanoAODSchema


In [ ]:
print(f"python: {sys.version}\n")

print(f"numpy:   {np.__version__}")
print(f"uproot:  {uproot.__version__}")
print(f"awkward: {ak.__version__}")
print(f"vector:  {vector.__version__}")
print()

print(f"coffea:  {coffea.__version__}")



In [ ]:
data_dir = '/home/bellis/top_data/NANOAOD/'

# Laptop
#infilename = 'small_skims_10k/TT_TToBCE_TuneCP5_BNV_2018_SMALL_10k.root'
#infilename = 'TTToHadronic_UL_2018_SMALL_100k.root'

# Beth Harmon
#infilename = 'small_skims_1k/TTbarPowheg_Hadronic_2017_SMALL_1k.root'
#infilename = 'TTToSemiLeptonic_UL_2018.root' # Also desktop home computer
infilename = 'Reza_signal/nAOD_step_BNV_TT_TSUE/NAOD-00000_190.root' # Also desktop home computer

# Desktop home computer
#infilename = 'small_skims_1k/TTbarPowheg_Hadronic_2017_SMALL_1k.root'


infile = uproot.open(data_dir + infilename)


In [ ]:
nevents = infile["Events"].num_entries

print(nevents)

In [ ]:
#events = coffea.nanoevents.NanoEventsFactory.from_root(infilename, schemaclass=NanoAODSchema).events()


In [ ]:
#infile['Events'].keys()

In [ ]:
jet_branch_arrays = infile["Events"].arrays(filter_name="Jet_*")#[0:100]

mask_jet_pt = jet_branch_arrays['Jet_pt']>20

print(mask_jet_pt)
print(infile["Events"]["Jet_pt"].array())

print(infile["Events"]["Jet_pt"].array()[mask_jet_pt])


njet = infile["Events"]["nJet"].array()#[0:100]
njet_masked = ak.num(infile["Events"]["Jet_pt"].array()[mask_jet_pt])#[0:100]

print(njet_masked)

In [ ]:
plt.figure(figsize=(12,4))
plt.hist(njet, range=(0.5,20.5),bins=20)
plt.hist(njet_masked, range=(0.5,20.5),bins=20,alpha=0.5)

plt.gca().set_xticks(np.arange(0, 20, 2));

In [ ]:
min_njets = 5
max_njets = 7

mask_njets = (njet>=min_njets) & (njet<=max_njets)

jets = ak.zip({
    "pt": jet_branch_arrays[mask_njets]["Jet_pt"],
    "phi": jet_branch_arrays[mask_njets]["Jet_phi"],
    "eta": jet_branch_arrays[mask_njets]["Jet_eta"],
    "mass": jet_branch_arrays[mask_njets]["Jet_mass"],
    "btag": jet_branch_arrays[mask_njets]["Jet_btagDeepB"],
}, with_name="Momentum4D")

In [ ]:
had_combos = ak.combinations(jets, 5)
#quad_charge = quads["0"].charge + quads["1"].charge + quads["2"].charge + quads["3"].charge

pt_cut = 25

mask_pt = (had_combos["0"].pt>pt_cut) & (had_combos["1"].pt>pt_cut) & (had_combos["2"].pt>pt_cut) \
          & (had_combos["3"].pt>pt_cut) & (had_combos["4"].pt>pt_cut)


jet1, jet2, jet3, jet4, jet5 = ak.unzip(had_combos[mask_pt])

p4_had = jet1 + jet2 + jet3
p4_bnv = jet4 + jet5

#mask_pt = (jet1.pt>25) & (jet2.pt>25) & (jet3.pt>25) & (jet4.pt>25) & (jet5.pt>25)

mask_btag = (jet1.btag>0.5) | (jet2.btag>0.5) & (jet3.btag>0.5)

#njet_masked = infile["Events"]["nJet"].array()#[0:100]



In [ ]:
plt.figure()
plt.hist(ak.flatten(jet1.pt),bins=400,range=(0,400));
plt.hist(ak.flatten(jet2.pt),bins=400,range=(0,400),alpha=0.6);
plt.hist(ak.flatten(jet3.pt),bins=400,range=(0,400),alpha=0.5);
plt.hist(ak.flatten(jet4.pt),bins=400,range=(0,400),alpha=0.5);
plt.hist(ak.flatten(jet5.pt),bins=400,range=(0,400),alpha=0.5);



In [ ]:
topp4 = (jet1 + jet2 + jet3)#[mask_pt & mask_btag]

print(len(ak.flatten(topp4.mass)))

plt.hist(ak.flatten(topp4.mass), bins=100, range=(0, 1200));

In [ ]:
print(njet)
print(len(njet))

In [ ]:
dR12 = jet1.deltaR(jet2)
dR13 = jet1.deltaR(jet3)
dR23 = jet2.deltaR(jet3)

In [ ]:
plt.figure(figsize=(15,4))
plt.subplot(1,3,1)
plt.hist(ak.flatten(dR12), bins=100, range=(0, 6.3));

plt.subplot(1,3,2)
plt.hist(ak.flatten(dR13), bins=100, range=(0, 6.3));

plt.subplot(1,3,3)
plt.hist(ak.flatten(dR23), bins=100, range=(0, 6.3));


In [ ]:
#for m in topp4.mass:
#    print(m)

In [ ]:
plt.figure(figsize=(15,4))
plt.subplot(1,3,1)
plt.hist(ak.flatten((jet1 + jet2).mass), bins=100, range=(0, 600));

plt.subplot(1,3,2)
plt.hist(ak.flatten((jet1 + jet3).mass), bins=100, range=(0, 600));

plt.subplot(1,3,3)
plt.hist(ak.flatten((jet2 + jet3).mass), bins=100, range=(0, 600));

In [ ]:
print(jet1[3])

for p in jet1[3]:
    print(p)

In [ ]:
'''
0 (1, 2, 3, 4, 5)
1 2 3    4 5
1 2 4    3 5
1 2 5    3 4
1 3 4    2 5
1 3 5    2 4

1 4 5    2 3
2 3 4    1 5
2 3 5    1 4
2 4 5    1 3
3 4 5    1 2
'''

p4_hads = []
p4_bnvs = []

p4_had = jet1 + jet2 + jet3; p4_bnv = jet4 + jet5
p4_hads.append(p4_had); p4_bnvs.append(p4_bnv)

p4_had = jet1 + jet2 + jet4; p4_bnv = jet3 + jet5
p4_hads.append(p4_had); p4_bnvs.append(p4_bnv)

p4_had = jet1 + jet2 + jet5; p4_bnv = jet3 + jet4
p4_hads.append(p4_had); p4_bnvs.append(p4_bnv)

p4_had = jet1 + jet3 + jet4; p4_bnv = jet2 + jet5
p4_hads.append(p4_had); p4_bnvs.append(p4_bnv)

p4_had = jet1 + jet3 + jet5; p4_bnv = jet2 + jet4
p4_hads.append(p4_had); p4_bnvs.append(p4_bnv)
#
p4_had = jet1 + jet4 + jet5; p4_bnv = jet2 + jet3
p4_hads.append(p4_had); p4_bnvs.append(p4_bnv)

p4_had = jet2 + jet3 + jet4; p4_bnv = jet1 + jet5
p4_hads.append(p4_had); p4_bnvs.append(p4_bnv)

p4_had = jet2 + jet3 + jet5; p4_bnv = jet1 + jet4
p4_hads.append(p4_had); p4_bnvs.append(p4_bnv)

p4_had = jet2 + jet4 + jet5; p4_bnv = jet1 + jet3
p4_hads.append(p4_had); p4_bnvs.append(p4_bnv)

p4_had = jet3 + jet4 + jet5; p4_bnv = jet1 + jet2
p4_hads.append(p4_had); p4_bnvs.append(p4_bnv)


In [ ]:
had_masses = []
bnv_masses = []
for p in p4_hads:
    m = p.mass
    print(m)
    print(type(m))
    had_masses += ak.flatten(m).tolist()

In [ ]:
len(had_masses)

In [ ]:
had_masses[3]

In [ ]:
plt.hist(had_masses,bins=100,range=(0,600));

# Boost

In [ ]:
#jet1.boost_p4(topp4).deltaR(jet2.boost_p4(topp4))

https://vector.readthedocs.io/en/latest/api/vector._methods.html?highlight=boost

In [ ]:
x = vector.obj(pt=2, eta=1.5, phi=1, energy=10)
y = x.from_rhophietatau(rho=x.rho, eta=x.eta, phi=x.phi, tau=20)

print(f"{x.mass:6.3f}  {x.pt}  {x.eta}  {x.phi}  {x.energy:6.2f}")
print(f"{y.mass:6.3f}  {y.pt}  {y.eta}  {y.phi}  {y.energy:6.2f}")

#print(y.mass, y.pt, y.eta, y.phi, y.energy)

#print(x.energy, y.energy)



Need to use `tau` instead of `mass`.

Distinction between geometrical vectors, which have a minimum of attribute and method names, and vectors representing momentum, which have synonyms like pt = rho, energy = t, mass = tau.



https://vector.readthedocs.io/en/latest/index.html

https://stackoverflow.com/questions/72906846/can-i-recalculate-the-energy-of-an-awkward-array-of-vectors-by-declaring-a-new-m/72929486?noredirect=1#comment128822783_72929486

In [ ]:
topp4 = (jet1 + jet2 + jet3)
topp4 = ak.with_field(topp4, 173.0, "tau")

print(topp4.pt)
print(topp4.rho)

print(topp4.mass)
print(topp4.tau)

print(topp4.energy)
print(topp4.t)

In [ ]:
# Set to mass of top
topp4 = (jet1 + jet2 + jet3)
print(type(topp4))

topp4 = ak.with_field(topp4, 173.0, "tau")

#print(type(topp4))

#print(topp4.array())

#jet1_boosted = jet1.boost_p4(topp4)
#jet2_boosted = jet2.boost_p4(topp4)
#jet3_boosted = jet3.boost_p4(topp4)

jet1_boosted = jet1.boostCM_of(topp4)
jet2_boosted = jet2.boostCM_of(topp4)
jet3_boosted = jet3.boostCM_of(topp4)

In [ ]:
print(jet1_boosted.pt)
print(jet1.pt)

In [ ]:
dR12_boosted = jet1_boosted.deltaR(jet2_boosted)
dR13_boosted = jet1_boosted.deltaR(jet3_boosted)
dR23_boosted = jet2_boosted.deltaR(jet3_boosted)

In [ ]:
plt.figure(figsize=(15,4))
plt.subplot(1,3,1)
plt.hist(ak.flatten((jet1_boosted + jet2_boosted).mass), bins=100, range=(0, 600));

plt.subplot(1,3,2)
plt.hist(ak.flatten((jet1_boosted + jet3_boosted).mass), bins=100, range=(0, 600));

plt.subplot(1,3,3)
plt.hist(ak.flatten((jet2_boosted + jet3_boosted).mass), bins=100, range=(0, 600));

In [ ]:
plt.figure(figsize=(15,4))
plt.subplot(1,3,1)
plt.hist(ak.flatten(dR12_boosted), bins=100, range=(0, 6.3));

plt.subplot(1,3,2)
plt.hist(ak.flatten(dR13_boosted), bins=100, range=(0, 6.3));

plt.subplot(1,3,3)
plt.hist(ak.flatten(dR23_boosted), bins=100, range=(0, 6.3));


In [ ]:
jet3_boosted[0].phi

In [ ]:
dPhi12 = jet1_boosted.deltaphi(jet2_boosted)
dPhi13 = jet1_boosted.deltaphi(jet3_boosted)
dPhi23 = jet2_boosted.deltaphi(jet3_boosted)

dPhi1_23 = jet1_boosted.deltaphi(jet2_boosted + jet3_boosted)


dTheta12 = jet1_boosted.deltaangle(jet2_boosted)
dTheta13 = jet1_boosted.deltaangle(jet3_boosted)
dTheta23 = jet2_boosted.deltaangle(jet3_boosted)

dTheta1_23 = jet1_boosted.deltaangle(jet2_boosted + jet3_boosted)



In [ ]:
plt.figure(figsize=(15,8))
plt.subplot(2,2,1)
plt.hist(ak.flatten(dPhi12), bins=100, range=(-4, 4));

plt.subplot(2,2,2)
plt.hist(ak.flatten(dPhi13), bins=100, range=(-4, 4));

plt.subplot(2,2,3)
plt.hist(ak.flatten(dPhi23), bins=100, range=(-4, 4));

plt.subplot(2,2,4)
plt.hist(ak.flatten(dPhi1_23), bins=100, range=(-4, 4));


#'''
plt.figure(figsize=(15,8))
plt.subplot(2,2,1)
plt.hist(ak.flatten(dTheta12), bins=100, range=(-4, 4));

plt.subplot(2,2,2)
plt.hist(ak.flatten(dTheta13), bins=100, range=(-4, 4));

plt.subplot(2,2,3)
plt.hist(ak.flatten(dTheta23), bins=100, range=(-4, 4));

plt.subplot(2,2,4)
plt.hist(ak.flatten(dTheta1_23), bins=100, range=(-4, 4));

#''';

In [ ]:
mask_j1_gt_j2 = (jet1_boosted.pt > jet2_boosted.pt)

In [ ]:
plt.figure(figsize=(15,4))
plt.subplot(1,3,1)
plt.hist(ak.flatten(dPhi12[mask_j1_gt_j2]), bins=100, range=(-4, 4));

#plt.subplot(1,3,2)
#plt.hist(ak.flatten(dPhi13), bins=100, range=(-4, 4));

#plt.subplot(1,3,3)
#plt.hist(ak.flatten(dPhi23), bins=100, range=(-4, 4));



In [ ]:
jet1_boosted.deltaangle(jet2_boosted)

In [ ]:
#for p4 in topp4.boostCM_of(topp4):
#    print(p4)

# Function to do things

In [ ]:
def event_hypothesis(jets):
    had_p4 = jets[0] + jets[1] + jets[2]
    
    had_dR12 = jets[0].deltaR(jets[1])
    had_dR13 = jets[0].deltaR(jets[2])
    had_dR23 = jets[1].deltaR(jets[2])
    had_dR1_23 = jets[0].deltaR(jets[1] + jets[2])
    
    had_p12 = jets[0] + jets[1]
    had_p13 = jets[0] + jets[2]
    had_p23 = jets[1] + jets[2]
    
    # btag
    jets[0].btag
    jets[1].btag
    jets[2].btag

    # Boost
    boost_p4 = had_p4

    boost_p4 = ak.with_field(boost_p4, 173.0, "tau")

    jet1_boosted = jets[0].boostCM_of(boost_p4)
    jet2_boosted = jets[1].boostCM_of(boost_p4)
    jet3_boosted = jets[2].boostCM_of(boost_p4)
    
    had_dThetaCM12 = jet1_boosted.deltaangle(jet2_boosted)
    had_dThetaCM13 = jet1_boosted.deltaangle(jet3_boosted)
    had_dThetaCM23 = jet2_boosted.deltaangle(jet3_boosted)
    had_dThetaCM1_23 = jet1_boosted.deltaangle(jet2_boosted + jet3_boosted)
    
    return had_p4.mass, had_p12.mass, had_p13.mass, had_p23.mass, \
           had_dR12, had_dR13, had_dR23, had_dR1_23, \
           had_dThetaCM12, had_dThetaCM13, had_dThetaCM23, had_dThetaCM1_23, \
           jets[0].btag, jets[1].btag, jets[2].btag

    #p4_bnv = jets[3] + jets[4]

In [ ]:
min_njets = 5
max_njets = 9

mask_njets = (njet>=min_njets) & (njet<=max_njets)

jets = ak.zip({
    "pt": jet_branch_arrays[mask_njets]["Jet_pt"],
    "phi": jet_branch_arrays[mask_njets]["Jet_phi"],
    "eta": jet_branch_arrays[mask_njets]["Jet_eta"],
    "mass": jet_branch_arrays[mask_njets]["Jet_mass"],
    "btag": jet_branch_arrays[mask_njets]["Jet_btagDeepB"],
}, with_name="Momentum4D")

had_combos = ak.combinations(jets, 5)
#quad_charge = quads["0"].charge + quads["1"].charge + quads["2"].charge + quads["3"].charge

pt_cut = 25

mask_pt = (had_combos["0"].pt>pt_cut) & (had_combos["1"].pt>pt_cut) & (had_combos["2"].pt>pt_cut) \
          & (had_combos["3"].pt>pt_cut) & (had_combos["4"].pt>pt_cut)

jet1, jet2, jet3, jet4, jet5 = ak.unzip(had_combos[mask_pt])

values = event_hypothesis([jet1, jet2, jet3, jet4, jet5])

print(values)



In [ ]:
plt.figure(figsize=(16,12))

for i in range(len(values)):
    x = ak.flatten(values[i]).to_numpy()
    print(len(x),x)
    print(len(x[x==x]))
    x[x==-np.inf] = -999
    x[x==np.inf] = -999
    plt.subplot(5,5,i+1)
    plt.hist(x[x==x],bins=100)
    

# Create some mock vectors for test

In [ ]:
def pmag(p3):
    pmag = np.sqrt(p3[0]*p3[0] + p3[1]*p3[1] + p3[2]*p3[2])

    return pmag


################################################################################
# Lorentz boost
# 
# Test code

# pmom = [200, 90,50,50]
# rest_frame = pmom
# print(lorentz_boost(pmom,rest_frame))
#
################################################################################
def lorentz_boost(p4, rest_frame_p4, return_matrix=False, boost_matrix=None):

    # Let's assume p4 will be a 4-vector and rest_frame_p4 will be
    # a 4-element list [e,px,py,pz]

    #print("HERE!")
    #print(p4)
    #print(type(p4))
    pmom = p4[0:4]
    #pmom = [0,0,0,0]
    #pmom[0] = p4['e']
    #pmom[1] = p4['pz']
    #pmom[2] = p4['py']
    #pmom[3] = p4['pz']
    '''
    if type(p4) == list:
        pmom = list(p4)
    elif type(p4) == np.ndarray:
        pmom = list(p4[0:4].tolist())
    else:
        pmom[0] = p4['e']
        pmom[1] = p4['pz']
        pmom[2] = p4['py']
        pmom[3] = p4['pz']
    '''

    rest_frame = rest_frame_p4[0:4]
    #rest_frame = [0,0,0,0]
    #if type(rest_frame_p4) == list:
        #rest_frame = list(rest_frame_p4)
    #elif type(rest_frame_p4) == np.ndarray:
        #rest_frame = list(rest_frame_p4[0:4].tolist())
    #else:
        #rest_frame[0] = rest_frame_p4['e']
        #rest_frame[1] = rest_frame_p4['pz']
        #rest_frame[2] = rest_frame_p4['py']
        #rest_frame[3] = rest_frame_p4['pz']

    L = boost_matrix

    # If a matrix for the boost has not been passed in, then
    # calculate the matrix using the rest frame 4-vector
    if boost_matrix is None:
        p = rest_frame
        c = 1

        pm = pmag(p[1:4])
        #pmag = np.sqrt(p[1]*p[1] + p[2]*p[2] + p[3]*p[3])
        #E = np.sqrt((pmag*c)**2 + (m*c**2)**2)
        E = p[0]

        beta = pm/E
        betaX = p[1]/E
        betaY = p[2]/E
        betaZ = p[3]/E

        beta2 = beta*beta

        gamma = np.sqrt(1 / (1-beta2))

        gamma_minus_1 = gamma-1

        x = ((gamma_minus_1) * betaX) / beta2
        y = ((gamma_minus_1) * betaY) / beta2
        z = ((gamma_minus_1) * betaZ) / beta2

        L = np.matrix([[gamma,      -gamma*betaX, -gamma*betaY, -gamma*betaZ],
                    [-gamma*betaX,  1 + x*betaX,      x*betaY,      x*betaZ],
                    [-gamma*betaY,      y*betaX,  1 + y*betaY,      y*betaZ],
                    [-gamma*betaZ,      z*betaX,      z*betaY,  1 + z*betaZ]])

    if return_matrix is True:
        return L

    # Moving particle that will be boosted
    #vector = np.matrix([E,p[1],p[1],p[2]])
    vector = np.matrix(pmom)

    boosted_vec = L*np.matrix.transpose(vector)

    return boosted_vec

In [ ]:
pmom = [200, 90,50,50]
rest_frame = pmom
print(lorentz_boost(pmom,rest_frame))

# Question code

In [ ]:
import numpy as np
import matplotlib.pylab as plt

import uproot
import awkward as ak

import vector
vector.register_awkward()


In [ ]:
infile = uproot.open("/home/bellis/Downloads/SMHiggsToZZTo4L.root")

muon_branch_arrays = infile["Events"].arrays(filter_name="Muon_*")
electron_branch_arrays = infile["Events"].arrays(filter_name="Electron_*")

muons = ak.zip({
    "pt": muon_branch_arrays["Muon_pt"],
    "phi": muon_branch_arrays["Muon_phi"],
    "eta": muon_branch_arrays["Muon_eta"],
    "mass": muon_branch_arrays["Muon_mass"],
    "charge": muon_branch_arrays["Muon_charge"],
}, with_name="Momentum4D")

quads = ak.combinations(muons, 4)
mu1, mu2, mu3, mu4 = ak.unzip(quads)

p4 = mu1 + mu2 + mu3 + mu4

In [ ]:
print(type(p4))

In [ ]:
ncounter = 0

for m1,m2,m3,m4 in zip(mu1[3], mu2[3], mu3[3], mu4[3]):
    print("-----")
    print(m1)
    print(m2)
    print(m3)
    print(m4)
    ncounter += 1
    
print(f"ncounter: {ncounter}")

In [ ]:
p4['mass'] = 120

In [ ]:
p4 = mu1 + mu2 + mu3 + mu4

print(p4[3][0].energy, p4[3][0].mass)

p4 = ak.with_field(p4, 125.0, "mass")

print(p4[3][0].energy, p4[3][0].mass)

In [ ]:
p4['tau'] = 125

In [ ]:
p4 = mu1 + mu2 + mu3 + mu4
for p in p4[3]:
    print(p.energy)
print("-----")
p4 = ak.with_field(p4, 125.0, "tau")

for p in p4[3]:
    print(p.energy)

# Iterations

In [ ]:
from itertools import combinations

In [ ]:
x = combinations(np.arange(1,6),5)
for a,i in enumerate(x):
    print(a,i)
    print(i[0], i[1], i[2], "  ", i[3], i[4])
    print(i[0], i[1], i[3], "  ", i[2], i[4])
    print(i[0], i[1], i[4], "  ", i[2], i[3])
    print(i[0], i[2], i[3], "  ", i[1], i[4])
    print(i[0], i[2], i[4], "  ", i[1], i[3])
    print(i[0], i[3], i[4], "  ", i[1], i[2])
    print(i[1], i[2], i[3], "  ", i[0], i[4])
    print(i[1], i[2], i[4], "  ", i[0], i[3])
    print(i[1], i[3], i[4], "  ", i[0], i[2])
    print(i[2], i[3], i[4], "  ", i[0], i[1])

# Test jets and muons together

In [ ]:
infilename = '~/top_data/NANOAOD/Reza_signal/nAOD_step_BNV_TT_TSUE/NAOD-00000_190.root' # Also desktop home computer
#infilename = "~/Downloads/SMHiggsToZZTo4L.root"

infile = uproot.open(infilename)

muon_branch_arrays = infile["Events"].arrays(filter_name="Muon_*")
electron_branch_arrays = infile["Events"].arrays(filter_name="Electron_*")
jet_branch_arrays = infile["Events"].arrays(filter_name="Jet_*")#[0:100]

muons = ak.zip({
    "pt": muon_branch_arrays["Muon_pt"],
    "phi": muon_branch_arrays["Muon_phi"],
    "eta": muon_branch_arrays["Muon_eta"],
    "mass": muon_branch_arrays["Muon_mass"],
    #"charge": muon_branch_arrays["Muon_charge"],
}, with_name="Momentum4D")

electrons = ak.zip({
    "pt": electron_branch_arrays["Electron_pt"],
    "phi": electron_branch_arrays["Electron_phi"],
    "eta": electron_branch_arrays["Electron_eta"],
    "mass": electron_branch_arrays["Electron_mass"],
    #"charge": electron_branch_arrays["Electron_charge"],
}, with_name="Momentum4D")

jets = ak.zip({
    "pt": jet_branch_arrays["Jet_pt"],
    "phi": jet_branch_arrays["Jet_phi"],
    "eta": jet_branch_arrays["Jet_eta"],
    "mass": jet_branch_arrays["Jet_mass"],
    #"btag": jet_branch_arrays["Jet_btagDeepB"],
}, with_name="Momentum4D")


muons_plus = muons[muons.charge > 0]
muons_minus = muons[muons.charge < 0]
electrons_plus = electrons[electrons.charge > 0]
electrons_minus = electrons[electrons.charge < 0]


In [ ]:
mu1, mu2, e1, e2 = ak.unzip(ak.cartesian([muons_plus, muons_minus, electrons_plus, electrons_minus]))
plt.hist(ak.flatten((mu1 + mu2 + e1 + e2).mass), bins=100, range=(0, 200));

In [ ]:
had_combos = ak.combinations(jets, 5)
muon_combos = ak.combinations(muons, 1)

jet1, jet2, jet3, jet4, jet5 = ak.unzip(had_combos)
m1 = ak.unzip(muon_combos)

#mu1, mu2, e1, e2 = ak.unzip(ak.cartesian([muons_plus, muons_minus, electrons_plus, electrons_minus]))
#j1, j2, j3, j4, j5, muon = ak.unzip(ak.cartesian([had_combos, muon_combos]))
#t = ak.cartesian([had_combos, muon_combos])
j1, j2, j3, j4, j5, muon = ak.unzip(ak.cartesian([jet1,jet2,jet3,jet4,jet5,m1]))


In [ ]:
for i in t[3]:
    print(i)

In [ ]:
len(muon_combos)

In [ ]:
len(had_combos)

In [ ]:
had_combos

In [ ]:
muon_combos

# Question for Jim

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import uproot
import awkward as ak

import vector

import sys

vector.register_awkward()

In [ ]:
infilename = '/tmp/NAOD-00000_190.root'

infile = uproot.open(infilename)

muon_branch_arrays = infile["Events"].arrays(filter_name="Muon_*")
jet_branch_arrays = infile["Events"].arrays(filter_name="Jet_*")#[0:100]

muons = ak.zip({
    "pt": muon_branch_arrays["Muon_pt"],
    "phi": muon_branch_arrays["Muon_phi"],
    "eta": muon_branch_arrays["Muon_eta"],
    "mass": muon_branch_arrays["Muon_mass"],
    "charge": muon_branch_arrays["Muon_charge"],
}, with_name="Momentum4D")

jets = ak.zip({
    "pt": jet_branch_arrays["Jet_pt"],
    "phi": jet_branch_arrays["Jet_phi"],
    "eta": jet_branch_arrays["Jet_eta"],
    "mass": jet_branch_arrays["Jet_mass"],
    "btag": jet_branch_arrays["Jet_btagDeepB"],
}, with_name="Momentum4D")

jet_combos = ak.combinations(jets, 5)
muon_combos = ak.combinations(muons, 1)

jet1, jet2, jet3, jet4, jet5 = ak.unzip(jet_combos)
muon1 = ak.unzip(muon_combos)

In [ ]:
#x = ak.cartesian([jet_combos, muon_combos])

#j1, j2, j3, j4, j5, muon = ak.cartesian([jet_combos, muon_combos])

j1, j2, j3, j4, j5, muon = ak.unzip(ak.cartesian([jet1,jet2,jet3,jet4,jet5,muon1]))

In [ ]:
x[3]

for i in x[3]:
    print("----")
    print(i)

In [ ]:
#j1, j2, j3, j4, j5, muon = ak.unzip(ak.cartesian([jet1,jet2,jet3,jet4,jet5,m1]))